In [1]:
# Installing dependencies
!pip install unsloth
!pip install transformers==4.57.0
!pip install --no-deps trl==0.22.2
!pip install bert_score

  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
Using cached transformers-4.57.1-py3-none-any.whl (12.0 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.0
    Uninstalling transformers-4.57.0:
      Successfully uninstalled transformers-4.57.0
  Using cached transformers-4.57.0-py3-none-any.whl.metadata (41 kB)
Reason for being yanked: Error in the setup causing installation issues
Using cached transformers-4.57.0-py3-none-any.whl (12.0 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth 2025.11.3 requires transformers!=4.52.0,!=4.52.1,!=4.52.2,!=4.52.3,!=4.53.0,!=4.54.0,!=4.55.0,!=4.55.1,!=4.57.0,<=4.57.2,>=4.51.3

In [2]:
# @title
from datasets import load_dataset

ds = load_dataset("flaviagiammarino/vqa-rad")

In [3]:
# @title
tes_data  = ds["test"]
tes_data[0]
# @title
len_a = len(tes_data)

In [6]:
#  Inference using
from PIL import Image
from unsloth import FastVisionModel,FastLanguageModel   
from transformers import TextStreamer

tes_data  = ds["test"]
# tes_data
prediction = []
ground_truth = []
model_name = "lora_model_v02"
model, tokenizer = FastVisionModel.from_pretrained(
    model_name = model_name,
    load_in_4bit = True,
)

for i in range(len_a):
    question = tes_data[i]['question']
    answer   = tes_data[i]['answer']
    image_pt = tes_data[i]['image']   # PIL.Image object
    prompt = ' You are a briliant docter/teacher/radiologist answer the questions consisely and in short if the question is open ended then answer in a small correct sentence if it is closed ended then answer in 1-2 words like Yes , True ,False etc...'
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text",  "text": f"{prompt}\nQuestion: {question}"},
                {"type": "image", "image": image_pt}
            ]
        }
    ]
    image = [image_pt]

    input_text = tokenizer.apply_chat_template(messages,add_generation_prompt=True)
    input = tokenizer(
        image,
        input_text,
        return_tensors ="pt",
    ).to("cuda")


    from transformers import TextStreamer
    FastLanguageModel.for_inference(model)
    textstreamer = TextStreamer(tokenizer)
    a = model.generate(**input,streamer=textstreamer,max_new_tokens=200,temperature=1)
    prediction.append(a)
    ground_truth.append(answer)


==((====))==  Unsloth 2025.11.3: Fast Qwen3_Vl patching. Transformers: 4.57.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/213 [00:00<?, ?B/s]

<|im_start|>user
 You are a briliant docter/teacher/radiologist answer the questions consisely and in short if the question is open ended then answer in a small correct sentence if it is closed ended then answer in 1-2 words like Yes , True ,False etc...
Question: is there evidence of an aortic aneurysm?<|vision_start|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|i

In [7]:
len(prediction)
len(ground_truth)
# prediction[0]
# ground_truth[0]

451

In [5]:
ground_truth = tes_data[:452]['answer']

In [8]:
import json 
with open ('answersv02.json','r') as f:
    predic = json.load(f)
print(predic)

['yes', 'no', 'no', 'left', 'right', 'no', 'left', 'right', 'smaller', 'yes', 'yes', 'bilateral temporal lobe infarct', 'air space', 'no', 'yes', 'no', 'chest x-ray', 'lymph nodes', 'yes', 'no', 'liver', 'no', 'axial', 'no', 'no', 'ct - noncontrast', 'no', 'no', 'falx cerebri', 'subependymal giant cell astrocytoma (sgca)', 'no', 'no', 'supratentorial', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'yes', 'fat', 'subcutaneous air', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'left atrium', 'right upper lung', 'no', 'no', 'no', 'yes', 'no', 'no', 'yes', 'no', 'no', 'no', 'yes', 'T1 C+', 'yes', 'no', 'malignant lung mass in the right middle lobe', 'pleural effusion', 'cardiovascular', 'yes', 'yes', 'no', 'left lung, lower lobe', 'right upper lobe', 'no', 'yes', 'pa', 'right upper quadrant', 'left adnexa', 'body', 'no', 't1w with contrast', 'left kidney', 'left frontal lobe', 'no', 'no', 'axial', 'mri', 'yes', 'lateral ventricles', 'abdomen', 'no', 'no', 'metastatic lesion', 'no', 'yes', 'right'

In [9]:
# @title
from bert_score import score
import torch

# Compute BERTScore
P, R, F1 = score(predic, ground_truth, lang="en", verbose=True, nthreads=10)

# Print formatted evaluation summary
print(f"""
╔══════════════════════════════════════════════════════════════════════════╗
║                         🔬 Model Evaluation Summary                      ║
╚══════════════════════════════════════════════════════════════════════════╝

🧠 Model Name: Mode_TestingV1 (lora_model) 
🏗️  Base Model: Qwen2-VL 7B   
📚 Finetuned Dataset: PubMed Vision (~6,666 datapoints)  
📉 Average Training Loss: 1.558  
🧪 Evaluation Dataset: flaviagiammarino/VQA-RAD (451 samples)  
💻 Training Hardware: NVIDIA T4 (Colab)  
⚡ Inference Hardware: NVIDIA L4 GPU  

────────────────────────────────────────────────────────────────────────────
📊 Final Average BERTScore (Mean of all {len(P)} Samples):
   • Precision : {P.mean():.4f}
   • Recall    : {R.mean():.4f}
   • F1 Score  : {F1.mean():.4f}
────────────────────────────────────────────────────────────────────────────
""")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/8 [00:00<?, ?it/s]

done in 7.93 seconds, 56.89 sentences/sec

╔══════════════════════════════════════════════════════════════════════════╗
║                         🔬 Model Evaluation Summary                      ║
╚══════════════════════════════════════════════════════════════════════════╝

🧠 Model Name: Mode_TestingV1 (lora_model) 
🏗️  Base Model: Qwen2-VL 7B   
📚 Finetuned Dataset: PubMed Vision (~6,666 datapoints)  
📉 Average Training Loss: 1.558  
🧪 Evaluation Dataset: flaviagiammarino/VQA-RAD (451 samples)  
💻 Training Hardware: NVIDIA T4 (Colab)  
⚡ Inference Hardware: NVIDIA L4 GPU  

────────────────────────────────────────────────────────────────────────────
📊 Final Average BERTScore (Mean of all 451 Samples):
   • Precision : 0.9497
   • Recall    : 0.9468
   • F1 Score  : 0.9479
────────────────────────────────────────────────────────────────────────────



In [10]:
# VQA scoring utilities — add below your BERTScore code
import re
import string
from collections import Counter
from bert_score import score as bert_score  # if reusing
from typing import List, Union, Iterable

def normalize_answer(s: str) -> str:
    """Lower, remove punctuation, articles and extra whitespace."""
    if s is None:
        return ""
    s = s.lower().strip()
    # replace punctuation with space
    s = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', s)
    # remove articles
    s = re.sub(r'\b(a|an|the)\b', ' ', s)
    # normalize whitespace
    s = re.sub(r'\s+', ' ', s).strip()
    return s

def tokenize(s: str) -> List[str]:
    return normalize_answer(s).split()

def exact_match_score(pred: str, truths: Union[str, Iterable[str]]) -> int:
    """Return 1 if normalized prediction exactly matches any of the truth references."""
    pred_n = normalize_answer(pred)
    if isinstance(truths, str):
        truths = [truths]
    for t in truths:
        if pred_n == normalize_answer(t):
            return 1
    return 0

def f1_score(pred: str, truth: str) -> float:
    """Compute token-level F1 between a single pred and a single truth."""
    pred_tokens = tokenize(pred)
    truth_tokens = tokenize(truth)
    if len(pred_tokens) == 0 and len(truth_tokens) == 0:
        return 1.0
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return 0.0
    common = Counter(pred_tokens) & Counter(truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0.0
    precision = num_same / len(pred_tokens)
    recall = num_same / len(truth_tokens)
    return 2 * precision * recall / (precision + recall)

def max_ref_f1(pred: str, truth_refs: Iterable[str]) -> float:
    """For a prediction against multiple reference truth strings, return max F1."""
    return max(f1_score(pred, t) for t in truth_refs)

def numeric_match(pred: str, truths: Iterable[str], tolerance: float = 1e-2) -> bool:
    """
    Try to match as numbers (floats/ints). If both pred and any truth parse to numbers,
    return True if |pred - truth| <= tolerance (absolute).
    Useful for measurements (e.g., "2.5 cm", "25 mm" you may need unit-normalization upstream).
    """
    def as_num(s):
        try:
            # keep only digits, dot and minus
            s_clean = re.sub(r'[^\d\.-]', '', s)
            return float(s_clean)
        except Exception:
            return None
    pred_num = as_num(pred)
    if pred_num is None:
        return False
    for t in truths:
        t_num = as_num(t)
        if t_num is None:
            continue
        if abs(pred_num - t_num) <= tolerance:
            return True
    return False

def compute_vqa_scores(preds: List[str],
                       gts: List[Union[str, List[str]]],
                       numeric_tolerance: float = 1e-2) -> dict:
    """
    preds: list of predicted answers (one per sample)
    gts:   list of ground-truth references (string or list of strings) (one per sample)
    Returns dict with aggregated metrics: EM, token-F1 (mean), BERTScore (if you compute separately), and distribution.
    """
    assert len(preds) == len(gts), "predictions and ground-truth lists must be same length"

    total = len(preds)
    em_count = 0
    f1_sum = 0.0
    em_with_numeric = 0

    # optionally keep per-sample details
    per_sample = []

    for pred, truth_refs in zip(preds, gts):
        if isinstance(truth_refs, str):
            truth_refs = [truth_refs]
        # 1) Exact match ignoring minor normalization differences
        em = exact_match_score(pred, truth_refs)

        # 2) Numeric tolerant exact match (if numeric present)
        num_em = numeric_match(pred, truth_refs, tolerance=numeric_tolerance)

        # 3) Token-level F1: take max over references
        f1 = max_ref_f1(pred, truth_refs)

        em_count += em
        f1_sum += f1
        em_with_numeric += 1 if (em or num_em) else 0

        per_sample.append({
            "pred": pred,
            "refs": truth_refs,
            "EM": em,
            "NUM_MATCH": bool(num_em),
            "F1": f1
        })

    em_score = em_count / total
    f1_score_avg = f1_sum / total
    em_numeric_adjusted = em_with_numeric / total

    return {
        "total_samples": total,
        "exact_match": em_score,              # strict normalized exact match
        "token_f1": f1_score_avg,            # averaged token-level F1
        "exact_match_plus_numeric": em_numeric_adjusted,  # EM OR numeric-tolerant match
        "per_sample": per_sample
    }

# -------------------------
# Example usage integrating BERTScore (you already compute P,R,F1):
# -------------------------
# preds and ground_truth must be lists of strings (same length)
# preds = [...] ; ground_truth = [...]
vqa_results = compute_vqa_scores(preds=predic, gts=ground_truth, numeric_tolerance=0.05)

# If you want to compute BERTScore across the same lists (you already do)
P, R, F1 = score(predic, ground_truth, lang="en", verbose=True, nthreads=10)

# Now print a combined summary
print(f"""
────────────────────────────────────────────────────────────────────────────
📊 VQA Evaluation (on {vqa_results['total_samples']} samples):
   • Exact Match (normalized)            : {vqa_results['exact_match']:.4f}
   • Token-level F1 (avg)                : {vqa_results['token_f1']:.4f}
   • Exact Match (with numeric tolerance): {vqa_results['exact_match_plus_numeric']:.4f}
   • BERTScore (avg F1)                  : {F1.mean():.4f}
────────────────────────────────────────────────────────────────────────────
""")

# Optional: inspect a few failure cases
failures = [s for s in vqa_results['per_sample'] if not (s["EM"] or s["NUM_MATCH"])]
print(f"Sample failures (up to 5):")
for s in failures[:5]:
    print(f"  PRED: {s['pred']!r}  REFS: {s['refs']}  F1: {s['F1']:.3f}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/8 [00:00<?, ?it/s]

done in 7.61 seconds, 59.27 sentences/sec

────────────────────────────────────────────────────────────────────────────
📊 VQA Evaluation (on 451 samples):
   • Exact Match (normalized)            : 0.4590
   • Token-level F1 (avg)                : 0.4929
   • Exact Match (with numeric tolerance): 0.4590
   • BERTScore (avg F1)                  : 0.9479
────────────────────────────────────────────────────────────────────────────

Sample failures (up to 5):
  PRED: 'no'  REFS: ['yes']  F1: 0.000
  PRED: 'left'  REFS: ['right']  F1: 0.000
  PRED: 'right'  REFS: ['not seen here']  F1: 0.000
  PRED: 'right'  REFS: ['left']  F1: 0.000
  PRED: 'yes'  REFS: ['no']  F1: 0.000
